In [65]:
import pandas as pd 
import numpy as np

In [66]:
table_1 = pd.read_excel('Тестовое задание.xlsx', sheet_name='Данные клиента') # Считываю 1-й лист

In [67]:
table_2 = pd.read_excel('Тестовое задание.xlsx', sheet_name='Наша база') # Считываю 2-й лист

# На листе "Наша база" хранится база наших сотрудников. Очистите данные в листе "Данные клиента" и подтяните табельный номер из "Наша база" в "Данные клиента".

In [68]:
table_1['ФИО'] = table_1['ФИО'].str.strip() # Убираю лишние пробелы

In [69]:
table_1['ФИО'] = table_1['ФИО'].replace(to_replace =('ООО "Альфа Групп" ', 'Угли ', ' угли',  ' Огли'), value = '', regex = True) # Убрал Альфа банк

In [70]:
pivot_table_1 = pd.pivot_table(table_1, index=table_1['ФИО'], aggfunc='sum') # ссумирую повторяющиеся значения

In [71]:
pivot_table_1 = pivot_table_1.reset_index() # Убираю индекс

In [72]:
name = pivot_table_1['ФИО'].str.split(' ', expand=True) # хочу получить только имя и фамилию

In [73]:
pivot_table_1['Фамилия Имя'] = name[0] + ' ' + name[1] # добавляю имя и фвмилию

In [74]:
new_pivot = pd.pivot_table(pivot_table_1, index=pivot_table_1['Фамилия Имя'], values='Итого', aggfunc='sum') # Создаю сводную таблицу на основе 'Фамилия Имя'

In [75]:
new_pivot = new_pivot.reset_index() # Убираю индекс

In [76]:
basa_last_first_name = table_2['ФИО'].str.split(' ', expand=True)[[0, 1]] # Теперь нужно достать фамилию и имя из нашей базы

In [77]:
basa_last_first_name = basa_last_first_name[0] + ' ' + basa_last_first_name[1] # Объеденяю фамилию и имя 

In [78]:
basa_last_first_name = pd.DataFrame(basa_last_first_name).rename(columns={0 : 'Фамилия Имя'}) # Переименовываю

In [79]:
basa_last_first_name = basa_last_first_name.reset_index().rename(columns={'index' : 'Табельный номер'}) # Переименовываю

In [80]:
basa_last_first_name['Табельный номер'] = basa_last_first_name['Табельный номер'] + 1 # Актуализирую 'Табельный номер'

In [81]:
merge_1 = new_pivot.merge(basa_last_first_name, left_on=new_pivot['Фамилия Имя'], right_on=basa_last_first_name['Фамилия Имя'], how='left') # объеденяю таблицы по совпадениям

In [82]:
# Здесь я поступил следующим образом, т.к. есть несовпадения в таблице, 'Табельный номер' в некоторых местах пришлось проставить вручную, но все ровно будут пустые ячейки

In [83]:
merge_1.loc[4, 'Табельный номер'] = 4

In [84]:
merge_1.loc[0, 'Табельный номер'] = 6

In [85]:
merge_1.loc[12, 'Табельный номер'] = 13

In [86]:
merge_1.loc[14, 'Табельный номер'] = 14

In [87]:
merge_1.loc[19, 'Табельный номер'] = 86

In [88]:
merge_1.loc[26, 'Табельный номер'] = 12

In [89]:
merge_1.loc[27, 'Табельный номер'] = 5

In [90]:
merge_1['Табельный номер'] = merge_1['Табельный номер'].fillna(method='ffill').astype('int64') # дозаполняю пустые ячейки и меняю тип

C:\Users\Dexp\AppData\Local\Temp\ipykernel_7792\103420229.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merge_1['Табельный номер'] = merge_1['Табельный номер'].fillna(method='ffill').astype('int64') # дозаполняю пробелы и меняю тип


In [91]:
result = pd.pivot_table(merge_1, index=merge_1['Табельный номер'], values='Итого', aggfunc='sum') # ссумирую по 'Табельный номер'

In [95]:
result.reset_index() # вывод результата

,Табельный номер,Итого
0,1,3993.750000
1,2,860.000000
2,3,16144.959961
3,4,14408.270142
4,5,11135.519897
5,6,700.000000
6,7,59444.039551
7,8,53876.060059
8,9,44861.559082
9,10,29059.529785


In [97]:
print_result = result.to_excel('customer_data.xlsx', index=True)